In [1]:
import pandas as pd
from alphalens.utils import get_clean_factor_and_forward_returns
from alphalens.tears import create_full_tear_sheet
from tools.get_data.get_data_h5 import *
import os

* Alphalens需要我们准备两种数据：因子值，股票价格
* 因子数据要求用日期（level0）标的代码（level 1）多重索引（MultiIndex Series, DataFrame也可以）  
![示例图片](.//reference//alphalens1.jpg)
* 价格数据与因子数据的格式不同，索引是时间，每一列是每只股票对应的收盘价：  
![示例图片](.//reference//alphalens2.webp)

In [2]:
listdir = os.listdir('Y:/DataBase_RQ_files/minbar/equities')
stocks_list = [eqt[:-3] for eqt in listdir if eqt.endswith('.h5')][:3]
factors = pd.DataFrame()
for std in stocks_list:
    df_min = get_data(symbol=std, freq="daybar", type="equities")[[
        "datetime", "volume", "close", "symbol"
    ]][-2000:]
    df_min.set_index(["datetime", "symbol"], inplace=True, drop=True)
    factors = pd.concat([factors, df_min], axis=0)
rtn = factors.pivot_table(index='datetime', columns='symbol', values='close')

print(factors.head(),rtn.head())

                              volume   close
datetime   symbol                           
2014-05-07 000001.XSHE  4.180210e+07  5.6645
2014-05-08 000001.XSHE  6.559849e+07  5.7063
2014-05-09 000001.XSHE  5.562689e+07  5.7689
2014-05-12 000001.XSHE  1.040242e+08  5.9203
2014-05-13 000001.XSHE  5.284799e+07  5.8629 symbol      000001.XSHE  000002.XSHE
datetime                            
2014-05-07       5.6645       5.6537
2014-05-08       5.7063       5.5704
2014-05-09       5.7689       5.5022
2014-05-12       5.9203       5.5476
2014-05-13       5.8629       5.7068


In [3]:
factors.index.get_level_values('datetime')

DatetimeIndex(['2014-05-07', '2014-05-08', '2014-05-09', '2014-05-12',
               '2014-05-13', '2014-05-14', '2014-05-15', '2014-05-16',
               '2014-05-19', '2014-05-20',
               ...
               '2022-07-07', '2022-07-08', '2022-07-11', '2022-07-12',
               '2022-07-13', '2022-07-14', '2022-07-15', '2022-07-18',
               '2022-07-19', '2022-07-20'],
              dtype='datetime64[ns]', name='datetime', length=4000, freq=None)

In [4]:
merged_data = get_clean_factor_and_forward_returns(factor=factors["volume"],
                                                   prices=rtn,
                                                   # quantiles=5,
                                                   bins=10,
                                                   quantiles=None,
                                                   periods=(1,3,5, 10,20,30,40),
                                                   )
merged_data

c:\veighna_studio\lib\site-packages\alphalens\utils.py:319: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  delta_days = period_len.components.days - mode(days_diffs).mode[0]


Dropped 2.0% entries from factor data: 2.0% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


,,1D,3D,5D,10D,20D,30D,40D,factor,factor_quantile
date,asset,,,,,,,,,
2014-05-07,000001.XSHE,0.007379,0.045158,0.042404,0.044240,0.053456,0.096549,0.116727,4.180210e+07,1
2014-05-08,000001.XSHE,0.010970,0.027443,0.023781,0.042076,0.041165,0.096315,0.102991,6.559849e+07,1
2014-05-09,000001.XSHE,0.026244,0.023540,0.019917,0.047964,0.038916,0.075612,0.085510,5.562689e+07,1
2014-05-12,000001.XSHE,-0.009695,-0.013226,-0.006165,0.021164,0.036164,0.051315,0.065267,1.040242e+08,10
2014-05-13,000001.XSHE,0.007130,0.003565,0.000887,0.031162,0.048969,0.058367,0.045370,5.284799e+07,1
...,...,...,...,...,...,...,...,...,...,...
2022-05-18,000002.XSHE,0.022848,-0.024973,-0.031881,-0.049947,-0.034006,0.089267,-0.022848,7.611578e+07,1
2022-05-19,000002.XSHE,-0.018182,-0.058701,-0.044675,-0.080000,-0.057143,0.059740,-0.084675,1.017552e+08,10
2022-05-20,000002.XSHE,-0.029101,-0.035979,-0.040212,-0.085185,-0.016931,0.069312,-0.052910,1.602077e+08,10


In [5]:
$pip install jupyter

SyntaxError: invalid syntax (307970794.py, line 1)

In [ ]:
create_full_tear_sheet(merged_data, long_short=True, group_neutral=False, by_group=False)

In [ ]:
returns = rtn
out = np.empty_like(rtn)
out


In [ ]:
help(create_full_tear_sheet)
help(get_clean_factor_and_forward_returns)

In [ ]:
equities1 = get_data(symbol="000638.XSHE", freq="minbar", type="equities")
# rb99.to_feather('rb99.feather')
equities1.to_feather('equities1.feather')


In [ ]:
pd.read_feather("E:\\Work\\5 Research\\factor_analysis\\equities1.feather")

In [ ]:
rtn["000001.XSHE"].pct_change().cumsum().plot()

In [ ]:
(rtn["000001.XSHE"].pct_change()+1).cumprod().plot()

In [ ]:
rtn["000001.XSHE"]